In [ ]:
import os
import sys
import django

sys.path.append(
os.path.dirname(os.getcwd()))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "advance_django.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
django.setup()

In [8]:
from orm.models import Author, Book, Review

a1 = Author.objects.create(name="User 1")
a2 = Author.objects.create(name="User 2")

b1 = Book.objects.create(author=a1, title="Hello World")
b2 = Book.objects.create(author=a2, title="Nemesis")

r1 = Review.objects.create(book=b1, rating=3, comment="Nice One")
r2 = Review.objects.create(book=b2, rating=5, comment="Great One")




In [44]:
from django.db.models import FilteredRelation, Q, Count, F

books_with_high_rated_views = (
    Book.objects.annotate(
        high_review_books=FilteredRelation("review", condition=Q(review__rating__gte=5))
    )
    .filter(high_review_books__isnull=False)
    .values("title", book_review=F("high_review_books__rating"))
)

In [69]:
result = (
    Book.objects.annotate(
        high_rated_reviews=FilteredRelation(
            "review", condition=Q(review__rating__gte=4)
        ),
        low_rated_reviews=FilteredRelation(
            "review", condition=Q(review__rating__lt=4)
        ),
    )
    .filter(Q(high_rated_reviews__isnull=False) | Q(low_rated_reviews__isnull=False))
    .values("title", "high_rated_reviews__rating", "low_rated_reviews__rating")
)
print(result)

<QuerySet [{'title': 'Hello World', 'high_rated_reviews__rating': None, 'low_rated_reviews__rating': 3}, {'title': 'Nemesis', 'high_rated_reviews__rating': 5, 'low_rated_reviews__rating': None}]>


In [60]:
result.query.__str__()

'SELECT "orm_book"."title", COUNT(high_rated_reviews."id") AS "high_review_count" FROM "orm_book" INNER JOIN "orm_review" high_rated_reviews ON ("orm_book"."id" = high_rated_reviews."book_id" AND (high_rated_reviews."rating" >= 4)) WHERE high_rated_reviews."id" IS NOT NULL GROUP BY "orm_book"."id", "orm_book"."title", "orm_book"."author_id"'

In [45]:
books_with_high_rated_views

<QuerySet [{'title': 'Nemesis', 'book_review': 5}]>

In [46]:
books_with_high_rated_views.query.__str__()

'SELECT "orm_book"."title", high_review_books."rating" AS "book_review" FROM "orm_book" INNER JOIN "orm_review" high_review_books ON ("orm_book"."id" = high_review_books."book_id" AND (high_review_books."rating" >= 5)) WHERE high_review_books."id" IS NOT NULL'